In [1]:
#Imports
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import os
import pandas as pd
#import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist

ModuleNotFoundError: No module named 'splinter'

# Scrape 1 : title and text

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

browser.visit(url)

In [ ]:
html = browser.html
marsnasagov_soup = bs(html, 'html.parser')

In [ ]:
#fetch lists of HTML data
results_title = marsnasagov_soup.find_all('div',class_="content_title")
results_p = marsnasagov_soup.find_all('div', class_='rollover_description_inner')
#Check fetch
display(len(results_title),
        len(results_p))

In [ ]:
#Init empty lists
titles_list = []
ptext_list = []

In [ ]:
#grab all paragraphs
for paragraph in results_p:
    ptext_list.append(paragraph.text)

#grab all titles
for title in results_title:
    titles_list.append(title.text)

In [ ]:
#locate most recent
latest_title = titles_list[0]
latest_paragraph = ptext_list[0]

# Scrape 2: featured image (jpg)


In [ ]:
# Chrome driver and Splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
#Splinter navigate to site, find the image url for the current Featured Mars Image
browser.visit(url)
browser.click_link_by_id('full_image')

In [ ]:
#fetch HTML for list creation
html = browser.html
soup = bs(html, 'html.parser')
featured_image_url_find = soup.find('img', class_ = 'fancybox-image')
featured_image_url_find

In [ ]:
#combines root of url with new found 'Featured image'
featured_image_url = 'https://www.jpl.nasa.gov' + featured_image_url_find['src']
featured_image_url

# Scrape 3: TWITTER


In [ ]:
# Chrome driver and Splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [ ]:
#fetch HTML for list creation
html = browser.html
soup = bs(html, 'html.parser')
#grab all tweets
tweet_container_list = soup.find_all('div', class_ = 'js-tweet-text-container')

#find latest tweet container
tweet_container_list[0]

In [ ]:
tweets_list = []
for tweet_container in tweet_container_list:
    tweets_list.append(tweet_container.p.text.replace('\n', ' ').split('hPapic.twitter.com/', 1)[0])
    
   
#Store tweet
mars_weather = tweets_list[0]
mars_weather

# Scrape 4 : mars facts

In [ ]:
#Pandas scrape facts table Diameter, Mass, etc.
url = "https://space-facts.com/mars/"
mars_facts_tables = pd.read_html(url)
#grab table from list
mars_facts_df = mars_facts_tables[0]
mars_facts_df.columns = ['Measurement', 'Value']

In [ ]:
mars_facts_df_indexed = mars_facts_df.set_index('Measurement')
mars_facts_df_indexed

In [ ]:
#Pandas convert data to HTML table string.
mars_facts_html = mars_facts_df_indexed.to_html()

In [ ]:
print(mars_facts_html)

# Scrape 5 : hemispheres

In [ ]:
# Chrome driver and Splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

### Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]

In [ ]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')
hemi_links_list_raw = soup.find_all('a', class_ = 'itemLink product-item')

In [ ]:
# init title and link lists 
hemi_links_title_list = []
hemi_links_href_list = []
# Identify and return link to listing
#link = result.a['href']
for link in hemi_links_list_raw:
    hemi_links_title_list.append(link.text)
    hemi_links_href_list.append(link['href'])

In [ ]:
display(
    hemi_links_list_raw[0],
    hemi_links_title_list,
    hemi_links_href_list
)

In [ ]:
#complete image URLs
hemi_full_url_list = []
url_root = 'https://astrogeology.usgs.gov'
for url_tail in hemi_links_href_list:
    hemi_full_url_list.append(url_root + url_tail)

In [ ]:
hemi_full_url_list

In [ ]:
hemi_jpg_list = []
#full resolution image urls
for url in hemi_full_url_list:
    response = requests.get(url)
    soup = bs(response.text,'html.parser')
    #fetch full image(class ="wide-image")
    wide_image = soup.find('img', class_ = 'wide-image')
    hemi_jpg_list.append(url_root + wide_image['src'])

In [ ]:
hemi_jpg_list[0]

In [ ]:
clean_titles_list = []
for title in hemi_links_title_list:
    clean_titles_list.append(title.split('Enhanced')[0].strip())

In [ ]:
clean_titles_list[0]

In [ ]:
hemi_dict_list = []
for i in range(len(clean_titles_list)):
    hemi_dict_list.append(dict(zip(clean_titles_list[i:i+1], hemi_jpg_list[i:i+1])))
3hemi_dict_list

In [ ]:
[{e[0], e[1]} for e in zip(a,b)]

In [ ]:
clean_titles_list[]]

In [ ]:
#####################################
##### FOR DB AND APP PORTION ########
#####################################

# Define database and collection
db = client.craigslist_db
collection = db.items

# URL of page to be scraped
url = 'https://newjersey.craigslist.org/search/sss?sort=rel&query=guitar'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('li', class_='result-row')

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('a', class_='result-title').text
        # Identify and return price of listing
        price = result.a.span.text
        # Identify and return link to listing
        link = result.a['href']

        # Run only if title, price, and link are available
        if (title and price and link):
            # Print results
            print('-------------')
            print(title)
            print(price)
            print(link)

            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'price': price,
                'url': link
            }
            collection.insert_one(post)

    except Exception as e:
        print(e)

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define database and collection
db = client.craigslist_db
collection = db.items